In [ ]:
import numpy as np
from keras.datasets import fashion_mnist 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
!pip install wandb -qU

In [ ]:
import wandb
wandb.login()
wandb.init(project="CS6910 Assignment 1")

**Loading Data**

In [ ]:
(X, y), (X_test, y_test) = fashion_mnist.load_data()

**Question 1 - Plotting MNIST Fashion Dataset**

In [ ]:
classes = ['T-shirt', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
fig, ax = plt.subplots(2,5,figsize=(20, 10))
ax = ax.reshape(-1)

for i in range(10):
  sample = np.array(np.where(y == i)).reshape(-1)
  sample = sample[3]
  image = X[sample]
  ax[i].set_title(classes[i])
  ax[i].imshow(image)
  
plt.show()
wandb.log({'Labels':fig})

**Data Pre-Processing**

In [ ]:
X = X.reshape(X.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

#One Hot Encoding for y
def one_hot_encode(y):
  encoded_array = np.zeros((y.size, y.max()+1), dtype=int)
  encoded_array[np.arange(y.size),y] = 1 
  return encoded_array

def softmax_to_label(softmax_output):
  max_index = np.argmax(softmax_output, axis = 0)
  return max_index


X = X/255
X_test = X_test/255
y = one_hot_encode(y)


X = X.T
X_test = X_test.T
y = y.T

**Activation Functions**

In [ ]:
def sigmoid(z):
    #print(-z)
    return 1 / (1 + np.exp(-(z)))

def tanh(z):
    return np.tanh(z)

def relu(z):
    return (z>0)*(z) + ((z<0)*(z)*0.01)

def softmax(x):
    # x = np.float128(x)
    temp = np.exp(x-np.max(x, axis = 0))
    fin = temp/temp.sum(axis = 0)
    
    return fin

**Weight Initialisation**

In [ ]:
def initialize_parameters(input_size, n, output_size, initialisation):
  if (initialisation == 'Random'):
    parameters = {}
    parameters['W'+str(1)] = np.random.randn(n[0],input_size)*0.01
    parameters['b'+str(1)] = np.random.randn(n[0],1)
    for i in range(1,len(n)):
        parameters['W'+str(i+1)] = np.random.randn(n[i],n[i-1])*0.01
        parameters['b'+str(i+1)] = np.random.randn(n[i],1)
    parameters['W'+str(len(n)+1)] = np.random.randn(output_size,n[-1])*0.01
    parameters['b'+str(len(n)+1)] = np.random.randn(output_size,1)

  elif (initialisation == 'Xavier'):
    parameters = {}
    m = np.sqrt(6)/(input_size+n[0])
    parameters['W'+str(1)] = np.random.uniform(-m,m, (n[0],input_size))
    parameters['b'+str(1)] = np.random.randn(n[0],1)
    for i in range(1,len(n)):
        m = np.sqrt(6)/(n[i-1]+n[i])
        parameters['W'+str(i+1)] = np.random.uniform(-m,m, (n[i],n[i-1]) )
        parameters['b'+str(i+1)] = np.random.randn(n[i],1)
    m = np.sqrt(6)/(output_size+n[-1])
    parameters['W'+str(len(n)+1)] = np.random.uniform(-m,m,(output_size,n[-1]))
    parameters['b'+str(len(n)+1)] = np.random.randn(output_size,1)

  return parameters

def initialize_parameters_zeros(input_size, n, output_size):
    parameters = {}
    parameters['W'+str(1)] = np.zeros((n[0],input_size))
    parameters['b'+str(1)] = np.zeros((n[0],1))
    for i in range(1,len(n)):
        parameters['W'+str(i+1)] = np.zeros((n[i],n[i-1]))
        parameters['b'+str(i+1)] = np.zeros((n[i],1))
    parameters['W'+str(len(n)+1)] = np.zeros((output_size,n[-1]))
    parameters['b'+str(len(n)+1)] = np.zeros((output_size,1))
    return parameters

**Forward Propagation**

In [ ]:
def linear(W, X, b, activation_func):
    #print(f"W Shape = {W.shape}, X Shape= {X.shape}, W= {W}, X = {X}, b = {b} " )
    h = np.matmul(W,X)+b
    if activation_func == 'sigmoid':
        #print(h)
        a = sigmoid(h)
    elif activation_func == 'relu':
        a = relu(h)
    elif activation_func == 'tanh':
        a = tanh(h)
    elif activation_func == 'softmax':
        a = softmax(h)
    return h,a

def ForwardPropagation(X, parameters, activation_func):
    layer_wise_outputs = {}
    layer_wise_outputs['h1'], layer_wise_outputs['a1'] = linear(parameters['W1'], X, parameters['b1'], activation_func[0])
    for i in range(1, (len(parameters)//2)):
        layer_wise_outputs['h'+str(i+1)], layer_wise_outputs['a'+str(i+1)] = linear(parameters['W'+str(i+1)],layer_wise_outputs['a'+str(i)],parameters['b'+str(i+1)], activation_func[i])
    return layer_wise_outputs

**Loss Functions and Accuracy**

In [ ]:
def MSELoss(Y, Y_pred):
    MSE = np.mean((Y - Y_pred) ** 2, axis = 1)
    MSE = np.mean(MSE)
    return MSE

def CrossEntropyLoss(Y, Y_pred):
    CE = [-Y[i] * np.log(Y_pred[i]) for i in range(len(Y_pred))]
    crossEntropy = np.mean(CE)
    return crossEntropy

def cost(Y, Y_pred, loss_func):
    if (loss_func == 'MSE'):
        return (MSELoss(Y, Y_pred))
    elif (loss_func == 'CE'):
        return (CrossEntropyLoss(Y, Y_pred))

def accuracy_score(y_true, y_pred):
  correct = np.sum(y_true == y_pred)
  total = len(y_true)
  accuracy = correct / total
  return accuracy


**Back Propagation**

In [ ]:
def ActivationBackward(dA, Z, activation_func) :
    
    if (activation_func == 'sigmoid'):
        grad = sigmoid(Z)*(1-sigmoid(Z))
       
    elif (activation_func == 'relu'):
        grad = np.where(Z>0, 1, 0)
        
    elif (activation_func == 'tanh'):
        grad = 1 - tanh(Z)**2
    elif (activation_func == 'softmax'):
        grad = softmax(Z) * (1-softmax(Z))
    dZ = dA * grad
    return dZ

def softmax_derivative(x):
    return softmax(x) * (1-softmax(x))        
    
def LayerBackward(dZl, Wl, bl, A_prev):
    
    m = A_prev.shape[1]
    # print(m)
    dWl = (1/m) * np.matmul(dZl, A_prev.T)
    dbl = (1/m)* np.sum(dZl, axis=1, keepdims=True)
    dA_prev = np.matmul(Wl.T,dZl)
    
    assert (dA_prev.shape == A_prev.shape)
    assert (dWl.shape == Wl.shape)
    assert (dbl.shape == bl.shape)
    return dWl, dbl, dA_prev
   
def BackPropagate(parameters, layer_wise_outputs,X, Y, activation_func, loss):
    gradients = {}
    l = len(layer_wise_outputs)//2
    m = Y.shape[1]
    AL = layer_wise_outputs['a'+str(l)]
    HL = layer_wise_outputs['h'+str(l)]
    
    if loss == 'CE':
        gradients['dh'+str(l)] = AL-Y
    elif loss == 'MSE':
        gradients['dh'+str(l)] = (AL-Y) * softmax_derivative(HL)
        
    for i in range(l-1,0,-1):
        gradients['dW'+str(i+1)],gradients['db'+str(i+1)],gradients['da'+str(i)] = LayerBackward(gradients['dh'+str(i+1)], parameters['W'+str(i+1)], parameters['b'+str(i+1)], layer_wise_outputs['a'+str(i)])
        gradients['dh'+str(i)] = ActivationBackward(gradients['da'+str(i)], layer_wise_outputs['h'+ str(i)] , activation_func[i-1])
        
    gradients['dW'+str(1)],gradients['db'+str(1)],gradients['da'+str(0)] = LayerBackward(gradients['dh'+str(1)], parameters['W'+str(1)], parameters['b'+str(1)], X)    
    
    return gradients


# parameters = initialize_parameters(2, [1,2,3], 2)
# activation_func = ['sigmoid','sigmoid','sigmoid','softmax']
# X = np.array([1,2]).reshape(2,1)  
# Y = np.array([1,2]).reshape(2,1)
# loss = 'CE'

# layer_wise_outputs = ForwardPropagation(X, parameters, activation_func)
# print(layer_wise_outputs)
# print(BackPropagate(parameters, layer_wise_outputs, X, Y, activation_func, loss))
# print(parameters['W3'].shape)
# print(BackPropagate(parameters, layer_wise_outputs, X, Y, activation_func, loss)['dW3'].shape)

**Optimisers**

In [ ]:
class Optimiser:
    def __init__(self, lr):
        self.lr = lr

    def update(self, parameters, gradients):
        raise NotImplementedError

class SGD(Optimiser):
    def __init__(self, lr):
        super().__init__(lr)

    def update(self, parameters, gradients):
        L = len(parameters) // 2 
        for l in range(1, L + 1):
          parameters["W" + str(l)] = parameters["W" + str(l)] - learning_rate * gradients["dW" + str(l)]
          parameters["b" + str(l)] = parameters["b" + str(l)] - learning_rate * gradients["db" + str(l)]
        return parameters

class Momentum(Optimiser):
    def __init__(self, lr, beta):
        super().__init__(lr)
        self.beta = beta
        self.v = {}
  

    def update(self, parameters, gradients):
        L = len(parameters)//2
        if self.v == {}:
          for l in range(1, L + 1):
            self.v["W"+str(l)] = 0
            self.v["b"+str(l)] = 0
        for l in range(1, L + 1):
          self.v["W"+str(l)] = self.beta * self.v["W"+str(l)] + (1) * gradients["dW" + str(l)]
          parameters["W" + str(l)] = parameters["W" + str(l)] - self.lr * self.v["W"+str(l)]
          self.v["b"+str(l)] = self.beta * self.v["b"+str(l)] + (1) * gradients["db" + str(l)]
          parameters["b" + str(l)] = parameters["b" + str(l)] - self.lr * self.v["b"+str(l)]
        return parameters

class Nesterov(Optimiser):
    def __init__(self, lr, gamma):
        super().__init__(lr)
        self.gamma = gamma
        self.look_ahead = {}
        self.v = {}
        

    def update(self, parameters, gradients):
        L = len(parameters)//2
        if self.v == {}:
          for l in range(1, L + 1):
            self.v["W"+str(l)] = 0
            self.v["b"+str(l)] = 0
        if self.look_ahead == {}:
          for l in range(1, L + 1):
            self.look_ahead["W"+str(l)] = 0
            self.look_ahead["b"+str(l)] = 0

        for l in range(1, L + 1):
          self.look_ahead["W"+str(l)] = parameters["W" + str(l)]-self.gamma*self.v["W" + str(l)]
          parameters["W" + str(l)] = self.look_ahead["W"+str(l)] - self.lr * gradients["dW" + str(l)]
          self.v["W"+str(l)] = self.gamma * self.v["W"+str(l)] + self.lr * gradients["dW" + str(l)]

          self.look_ahead["b"+str(l)] = parameters["b" + str(l)]-self.gamma*self.v["b" + str(l)]
          parameters["b" + str(l)] = self.look_ahead["b"+str(l)] - self.lr * gradients["db" + str(l)]
          self.v["b"+str(l)] = self.gamma * self.v["b"+str(l)] + self.lr * gradients["db" + str(l)]

          
        return parameters

class RMSprop(Optimiser):
    def __init__(self, lr, decay_rate, eps):
        super().__init__(lr)
        self.decay_rate = decay_rate
        self.eps = eps
        self.s = {}


    def update(self, parameters, gradients):
        L = len(parameters)//2
        if self.s == {}:
          for l in range(1, L + 1):
            self.s["W"+str(l)] = 0
            self.s["b"+str(l)] = 0
        for l in range(1, L + 1):
          self.s["W"+str(l)] = self.decay_rate * self.s["W"+str(l)] + (1 - self.decay_rate) * (gradients["dW" + str(l)]**2)
          parameters["W" + str(l)] = parameters["W" + str(l)] - self.lr * gradients["dW" + str(l)] / (np.sqrt(self.s["W"+str(l)]) + self.eps)

          self.s["b"+str(l)] = self.decay_rate * self.s["b"+str(l)] + (1 - self.decay_rate) * (gradients["db" + str(l)]**2)
          parameters["b" + str(l)] = parameters["b" + str(l)] - self.lr * gradients["db" + str(l)] / (np.sqrt(self.s["b"+str(l)]) + self.eps)
        
        return parameters
        

class Adam(Optimiser):
    def __init__(self, lr, beta1, beta2, eps):
        super().__init__(lr)
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps
        self.m = {}
        self.v = {}
        self.t = 0

    def update(self, parameters, gradients):
        L = len(parameters)//2
        if self.m == {}:
          for l in range(1, L + 1):
            self.m["W"+str(l)] = 0
            self.m["b"+str(l)] = 0
        if self.v == {}:
          for l in range(1, L + 1):
            self.v["W"+str(l)] = 0
            self.v["b"+str(l)] = 0
        self.t += 1

        for l in range(1, L + 1):
          self.m["W" + str(l)] = self.beta1 * self.m["W" + str(l)] + (1 - self.beta1) * gradients["dW" + str(l)]
          self.v["W" + str(l)] = self.beta2 * self.v["W" + str(l)] + (1 - self.beta2) * (gradients["dW" + str(l)]**2)
          m_hat = self.m["W" + str(l)] / (1 - self.beta1 ** self.t)
          v_hat = self.v["W" + str(l)] / (1 - self.beta2 ** self.t)
          parameters["W" + str(l)] = parameters["W" + str(l)] - self.lr * m_hat / (np.sqrt(v_hat) + self.eps)

          self.m["b" + str(l)] = self.beta1 * self.m["b" + str(l)] + (1 - self.beta1) * gradients["db" + str(l)]
          self.v["b" + str(l)] = self.beta2 * self.v["b" + str(l)] + (1 - self.beta2) * (gradients["db" + str(l)]**2)
          m_hat = self.m["b" + str(l)] / (1 - self.beta1 ** self.t)
          v_hat = self.v["b" + str(l)] / (1 - self.beta2 ** self.t)
          parameters["b" + str(l)] = parameters["b" + str(l)] - self.lr * m_hat / (np.sqrt(v_hat) + self.eps)
        
        
        return parameters

 
class Nadam(Optimiser):
    def __init__(self, lr, beta1, beta2, eps):
        super().__init__(lr)
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps
        self.m = {}
        self.v = {}
        self.t = 0

    def update(self, parameters, gradients):
        L = len(parameters)//2
        if self.m == {}:
          for l in range(1, L + 1):
            self.m["W"+str(l)] = 0
            self.m["b"+str(l)] = 0
        if self.v == {}:
          for l in range(1, L + 1):
            self.v["W"+str(l)] = 0
            self.v["b"+str(l)] = 0
        self.t += 1

        for l in range(1, L + 1):
          self.m["W" + str(l)] = self.beta1 * self.m["W" + str(l)] + (1 - self.beta1) * gradients["dW" + str(l)]
          self.v["W" + str(l)] = self.beta2 * self.v["W" + str(l)] + (1 - self.beta2) * (gradients["dW" + str(l)]**2)
          m_hat = self.m["W" + str(l)] / (1 - self.beta1 ** self.t)
          v_hat = self.v["W" + str(l)] / (1 - self.beta2 ** self.t)
          m_hat_fin = (self.beta1*m_hat)+(1-self.beta1)*gradients["dW" + str(l)]/(1-(self.beta1)**self.t)
          parameters["W" + str(l)] = parameters["W" + str(l)] - self.lr * (m_hat_fin) / (np.sqrt(v_hat) + self.eps)

          self.m["b" + str(l)] = self.beta1 * self.m["b" + str(l)] + (1 - self.beta1) * gradients["db" + str(l)]
          self.v["b" + str(l)] = self.beta2 * self.v["b" + str(l)] + (1 - self.beta2) * (gradients["db" + str(l)]**2)
          m_hat = self.m["b" + str(l)] / (1 - self.beta1 ** self.t)
          v_hat = self.v["b" + str(l)] / (1 - self.beta2 ** self.t)
          m_hat_fin = (self.beta1*m_hat)+(1-self.beta1)*gradients["db" + str(l)]/(1-(self.beta1)**self.t)
          parameters["b" + str(l)] = parameters["b" + str(l)] - self.lr * (m_hat_fin) / (np.sqrt(v_hat) + self.eps)
        
        
        return parameters

**Training Our Model on Fashion-MNIST**


In [ ]:
# #Model Architechture

# n = [784,[64,32],10]
# activation_func = ['sigmoid','sigmoid','softmax']
# loss = 'MSE'
# batch_size = 200
# learning_rate = 0.001
# # optimiser = SGD(lr = learning_rate)
# # optimiser = Momentum(lr = learning_rate, beta = 0.2)
# # optimiser = Nesterov(lr = learning_rate, gamma = 0.9)
# # optimiser = RMSprop(lr = learning_rate, decay_rate = 0.1,eps = 1e-6)
# # optimiser = Adam(lr = learning_rate, beta1 = 0.9, beta2 = 0.99 ,eps = 1e-6)
# optimiser = Nadam(lr = learning_rate, beta1 = 0.9, beta2 = 0.99 ,eps = 1e-6)


# epochs = 30
# m = X.shape[1]
# parameters = initialize_parameters(n[0],n[1],n[2], 'Random')
# # parameters = parameters_test2
# count = 0

# #loss array 
# # losses = np.array([])

# while count < epochs :
#   training_loss = 0
#   count = count+1
#   l = len(parameters)//2
#   for i in np.arange(0, X.shape[1], batch_size):
#     batch_count = batch_size
#     if i + batch_size > X.shape[1]:
#       batch_count = X.shape[1] - i + 1
#     batch_size = batch_count

#     layer_wise_outputs = ForwardPropagation(X[:,i:i+batch_size], parameters, activation_func)  
#     gradients = BackPropagate(parameters, layer_wise_outputs, X[:,i:i+batch_size], y[:,i:i+batch_size], activation_func, loss)
#     parameters = optimiser.update(parameters, gradients)
#     training_loss = training_loss + cost(y[:,i:i+batch_size], layer_wise_outputs['a'+str(l)],loss)
#   print("Loss after "+ str(count) +"th epoch =" +str(training_loss*(batch_size)/m))



# test_outputs = ForwardPropagation(X_test, parameters, activation_func)


def softmax_to_label(softmax_output):
  max_index = np.argmax(softmax_output, axis = 0)
  return max_index

# test_outputs['a'+str(len(parameters)//2)] = softmax_to_label(test_outputs['a'+str(len(parameters)//2)])


# def accuracy_score(y_true, y_pred):
#     correct = np.sum(y_true == y_pred)
#     total = len(y_true)
#     accuracy = correct / total
#     return accuracy

# # print(softmax_to_label(y_test.T).shape)
# # print(test_outputs['a'+str(len(n))].shape)
# y_test_fin = softmax_to_label(y_test.T)
# print(f"Test Accuracy = {100*accuracy_score( y_test_fin, test_outputs['a'+str(len(parameters)//2)])} %")

def accuracy_score(y_true, y_pred):
    pred_labels = np.argmax(y_pred, axis=0)
    return np.sum(pred_labels == y_true) / len(y_true)

**Neural Network Class**

In [ ]:

class Network():
  def __init__(self, N):
    self.n = N['n']
    self.parameters = initialize_parameters(self.n[0],self.n[1],self.n[2], N['initialisation'])
    self.activation_func = N['activation_func']
    self.l = len(self.parameters)//2

  def train(self,X,y,X_val,y_val,loss, batch_size, optimiser, epochs):
    m = X.shape[1]
    count = 0
    while(count < epochs):
      count = count+1
      training_loss = 0
      for i in np.arange(0, X.shape[1], batch_size):
        batch_count = batch_size
        if i + batch_size > X.shape[1]:
          batch_count = X.shape[1] - i + 1
        batch_size = batch_count
        layer_wise_outputs = ForwardPropagation(X[:,i:i+batch_size], self.parameters, self.activation_func)  
        gradients = BackPropagate(self.parameters, layer_wise_outputs, X[:,i:i+batch_size], y[:,i:i+batch_size], self.activation_func, loss)
        self.parameters = optimiser.update(self.parameters, gradients)
        training_loss = training_loss + cost(y[:,i:i+batch_size], layer_wise_outputs['a'+str(self.l)],loss)

      training_loss_fin = training_loss*(batch_size)/m
      print("Epoch:"+str(count))
      print("Training Loss after "+ str(count) +"th epoch =" +str(training_loss_fin))

      training_outputs = ForwardPropagation(X, self.parameters, self.activation_func)
      training_score = accuracy_score(softmax_to_label(y), training_outputs['a'+str(self.l)])
      print("Training score after "+ str(count) +"th epoch =" + str(100*training_score))

      validation_outputs = ForwardPropagation(X_val, self.parameters, self.activation_func)
      validation_loss = cost(y_val, validation_outputs['a'+str(self.l)],loss)
      print("Validation Loss after "+ str(count) +"th epoch =" + str(validation_loss))
      validation_score = accuracy_score(softmax_to_label(y_val), validation_outputs['a'+str(self.l)])
      print("Validation Score after "+ str(count) +"th epoch =" + str(100*validation_score))

      metrics = {"train loss": training_loss_fin, "train score": training_score , "val loss": validation_loss, "val score": validation_score}
      wandb.log(metrics)


  def val_test(self,X,Y):
    test_outputs = ForwardPropagation(X, self.parameters, self.activation_func)
    print(f"Val Accuracy = {100*accuracy_score(softmax_to_label(Y), test_outputs['a'+str(self.l)])} %")
    wandb.log({'Val Accuracy' : accuracy_score(softmax_to_label(Y), test_outputs['a'+str(self.l)])})
    
  def test(self,X,Y):
    test_outputs = ForwardPropagation(X, self.parameters, self.activation_func)
    print(f"Test Accuracy = {100*accuracy_score(Y, test_outputs['a'+str(self.l)])} %")
    wandb.log({'Test Accuracy' : accuracy_score(Y, test_outputs['a'+str(self.l)])})   

In [ ]:
X_train, X_val, y_train,y_val  = train_test_split(X.T, y.T, test_size=0.1)
X_train, X_val, y_train,y_val  = X_train.T, X_val.T, y_train.T,y_val.T 

N = {'n' : [784,[128,128,64],10],
     'activation_func' : ['relu','relu','relu','softmax'],
     'initialisation' : 'Random'
    }


batch_size = 128
epochs = 5
loss = 'CE'
learning_rate = 0.1
optimiser = SGD(lr = learning_rate)


network = Network(N)
network.train(X_train , y_train, X_val, y_val, loss, batch_size, optimiser, epochs)
network.test(X_test, y_test)

**Question 4 - Sweeping through Hyper-parameters** 

In [ ]:

X_train, X_val, y_train,y_val  = train_test_split(X.T, y.T, test_size=0.1)
X_train, X_val, y_train,y_val  = X_train.T, X_val.T, y_train.T,y_val.T 


sweep_config = {
    'method': 'random',
    'name' : 'Hyperparameter Tuning'
}

parameters_dict = {
    'optimiser': {
        'values': ['sgd', 'momentum', 'nesterov', 'rmsprop', 'adam', 'nadam']
        },
    'layer_size': {
        'values': [32,64,128]
        },
    'epochs': {
          'values': [5,10]
        },
    'hidden_layers': {
          'values': [3,4,5]
        },
    'learning_rate': {
          'values': [1e-3,1e-4]
        },
    'weight_initialisation': {
          'values': ['Random', 'Xavier']
        },
    'activation_functions': {
          'values': ['sigmoid', 'tanh', 'relu']
        },
    'batch_size': {
          'values': [16,32,64]
        },
    'weight_decay': {
          'values': [0,0.0005,0.5]
        }
    }

sweep_config['parameters'] = parameters_dict
sweep_id = wandb.sweep(sweep_config, project="CS6910 Assignment 1")


def build_optimiser(optimiser_str,learning_rate):
  if optimiser_str == 'sgd':
    optimiser = SGD(lr = learning_rate)
  elif optimiser_str == 'momentum':
    optimiser = Momentum(lr = learning_rate, beta = 0.2)
  elif optimiser_str == 'nesterov':
    optimiser = Nesterov(lr = learning_rate, gamma = 0.9)
  elif optimiser_str == 'rmsprop':
    optimiser = RMSprop(lr = learning_rate, decay_rate = 0.1,eps = 1e-6)
  elif optimiser_str == 'adam':
    optimiser = Adam(lr = learning_rate, beta1 = 0.9, beta2 = 0.99 ,eps = 1e-6)
  elif optimiser_str == 'nadam':
    optimiser = Nadam(lr = learning_rate, beta1 = 0.9, beta2 = 0.99 ,eps = 1e-6)
  return optimiser


def train_sweep(config=None):
  with wandb.init(config=config) as run:
    config = wandb.config

    # loader = build_dataset(config.batch_size)
    # network = build_network(config.fc_layer_size, config.dropout)
    # optimizer = build_optimizer(network, config.optimizer, config.learning_rate)
    N = {'n' : [784,config.hidden_layers*[config.layer_size],10],
          'activation_func' : config.hidden_layers*[str(config.activation_functions)]+['softmax'],
          'initialisation' : config.weight_initialisation
        } 

    exp_name = 'in_'+config.weight_initialisation+'_e_'+ str(config.epochs)+'_hl_'+str(config.hidden_layers)
    exp_name = exp_name + '_ls_'+ str(config.layer_size)
    exp_name = exp_name+'_bs_'+ str(config.batch_size)+'_ac_'+str(config.activation_functions)
    
    wandb.run.name = exp_name


    batch_size = config.batch_size
    epochs = config.epochs
    loss = 'CE'
    optimiser = build_optimiser(config.optimiser, config.learning_rate)

    network = Network(N)
    network.train(X_train , y_train, X_val, y_val, loss, batch_size, optimiser, epochs)
    network.val_test(X_val,y_val)
    network.test(X_test, y_test)

wandb.agent(sweep_id, train_sweep, count=5)
wandb.finish()